In [ ]:
import sys
import argparse
import time
import os
import csv
import pandas as pd
import requests
import re
from selenium.webdriver.support import expected_conditions as EC
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.support.ui import Select
from selenium.webdriver.common.by import By
from selenium.webdriver.support.wait import WebDriverWait
from selenium.webdriver.common.action_chains import ActionChains
import easyocr


def solve_math_captcha(text):
    """
    Extracts and solves the CAPTCHA if it's a mathematical expression.
    Handles addition, subtraction, and multiplication using 'x'.
    """
    text = text.strip()
    match = re.search(r"(\d+)\s*([+x-])\s*(\d+)", text)
    if match:
        num1, operator, num2 = match.groups()
        num1, num2 = int(num1), int(num2)
        if operator == "+":
            return num1 + num2
        elif operator == "-":
            return num1 - num2
        elif operator == "x":  # Handle multiplication
            return num1 * num2
    return text  # If no math expression is found, return original text


# Set download directory
download_folder = "/home/titaniumplayer492003/Code/LEGIT/swatimishra02/pdfs"
chrome_options = webdriver.ChromeOptions()
prefs = {"download.default_directory": download_folder}
chrome_options.add_experimental_option("prefs", prefs)

# Initialize the WebDriver
driver = webdriver.Chrome()

driver.get("https://judgments.ecourts.gov.in/pdfsearch/")
select = Select(driver.find_element("id", "fcourt_type"))
select.select_by_visible_text("High Court")
time.sleep(3)

# Solve CAPTCHA
max_retries = 5
for attempt in range(max_retries):
    element = driver.find_element("id", "captcha_image")
    screenshot = element.screenshot_as_png
    with open("hc_screenshot.png", "wb") as file:
        file.write(screenshot)

    reader = easyocr.Reader(["en"])
    result = reader.readtext("hc_screenshot.png")

    text = "".join([detection[1] for detection in result])
    text = re.sub(r"[^0-9+\-*/]", "", text)  # Remove any non-math characters
    captcha_solution = solve_math_captcha(text)  # Solve if math expression

    captcha_field = driver.find_element("id", "captcha")
    captcha_field.clear()
    captcha_field.send_keys(captcha_solution)

    search = driver.find_element("id", "main_search")
    time.sleep(2)
    search.click()
    time.sleep(3)

    if (
        "Invalid Captcha..!!!" in driver.page_source
        or "Captcha should be numeric..!" in driver.page_source
        or "Please enter captcha" in driver.page_source
    ):
        driver.find_element(By.CLASS_NAME, "btn-close").click()
        driver.find_element("xpath", '//*[@id="captcha_div"]/div[1]/a').click()
        print(f"Attempt {attempt + 1} - Invalid captcha. Retrying...")
    else:
        print("Captcha entered successfully.")
        break

time.sleep(5)
court = driver.find_element(By.XPATH, "/html/body/div[2]/nav/div/div/ul/li[1]")
court.click()
time.sleep(2)

select_court = Select(
    driver.find_element(
        By.XPATH, "/html/body/div[2]/nav/div/div/ul/li[1]/div/div[1]/select"
    )
)
select_court.select_by_visible_text("High Court of Meghalaya")
time.sleep(2)

driver.find_element(By.XPATH, "/html/body/div[2]/nav/div/div/div/div/button[1]").click()
time.sleep(3)

Neither CUDA nor MPS are available - defaulting to CPU. Note: This module is much faster with a GPU.


Captcha entered successfully.


KeyboardInterrupt: 